In [ ]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 22 20:09:58 2021

@author: ronguy
"""
import time 
start = time.process_time()

import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
# import matplotlib.patches as  mpatches
plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

from scipy import signal, stats
import seaborn as sns

from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

from lmfit import minimize, Parameters
from umap import UMAP

import scanpy as sc
import anndata

# import schist as scs

# import networkx as nx

# from castle.common import GraphDAG
# from castle.metrics import MetricsDAG
# from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank
# import notears.notears as notears

# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# import keras
# from keras import layers
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
# from statsmodels.regression.linear_model import OLS
# from statsmodels.stats.outliers_influence import variance_inflation_factor




from functions import *

%matplotlib inline
%load_ext autoreload
%autoreload 2


# Load and initialize

In [ ]:
NamesAll=['CD45',
 'H3',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3.3',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
 'Ir_DNA2',
 'Live_Dead']


EpiCols=[
 'H3',
 'H3K27me2',
 'H3K4me3',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'H3.3',
 'H3K64ac',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'H3K36me3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'H3S28p',
]

CellIden=[
 'CD45',
 'K5',
 'EpCam',
 'aSMA',
 'Vimentin',
 'ZEB1',
 'ER',
 'CD49f',
 'CD24',
 'GATA3',
 'CD44',
 'K8-18',
]


ToNorm=[
 'H3',
 'K5',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3.3',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'H3K36me3',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'Ki67',
 'K8-18',
 'H3S28p',
 ]

NMS=['CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:
# dir="~/Desktop/CyTOF_Breast/Kaplan_1st/"
# dir="~/Desktop/biology/breast_cancer/data/"
dir="C:/Users/yishai/Dropbox/CyTOF_Breast/Kaplan_1st/"
K1=pd.read_csv(dir+"BCK-01_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")
K2=pd.read_csv(dir+"BCK-02_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")


params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

K1=K1[NamesAll]
K2=K2[NamesAll]



# Gate on H3.3/H4 too low, but also remove outliers 99.99% from all 

In [ ]:
GateColumns=['H3.3','H4']#,'H3']#,'H3']


def Gate(data,name):
    ddf=data.copy()
    print(name)
    print("Initial ",len(ddf))
    ddf=ddf[(ddf[GateColumns]>5).all(axis=1)]
    print("Core Gate ",len(ddf))
    ddf=ddf[(ddf<np.quantile(ddf,0.9999,axis=0)).all(axis=1)]
    print("Outlier Gate ",len(ddf))
    data=ddf.copy()
    del ddf
    return data


K1=Gate(K1,"K1")
K2=Gate(K2,"K2")

In [ ]:

scFac=5
K1=np.arcsinh(K1/scFac)
K2=np.arcsinh(K2/scFac)


In [ ]:
for M in NamesAll:
    plt.figure()
    sns.kdeplot(K1[M],c='r',label='Tumor 1')
    sns.kdeplot(K2[M],c='g',label='Tumor 2')
   
    plt.legend()
    plt.title('ArcSinh Unnormalized')
    plt.savefig('Plots/Hist_'+M+'.png')
    plt.show()

In [ ]:
ThK1={}
for N in ['CD45']:
    kernel = stats.gaussian_kde(K1[N])
    r=np.linspace(K1[N].min(),5,100)
    inv_data = 1/kernel(r)
    # minima : use builtin function fo find (min) peaks (use inversed data)
    min_peakind = signal.find_peaks_cwt(inv_data, np.arange(0.02,20,0.001))

    print(min_peakind)
    plt.figure(figsize=(3,3))
    plt.plot(r,kernel(r))
    sns.kdeplot(K1[N])
    plt.scatter(r[min_peakind],kernel(r[min_peakind]))
#    plt.yscale('log')
    plt.show()
    # Q=input("Peak Num (0 based):")
    Q=0
    ThK1[N]=r[min_peakind[np.int(Q)]]

In [ ]:
ThK2={}
for N in ['CD45']:
    kernel = stats.gaussian_kde(K2[N])
    r=np.linspace(K2[N].min(),5,100)
    inv_data = 1/kernel(r)
    # minima : use builtin function fo find (min) peaks (use inversed data)
    min_peakind = signal.find_peaks_cwt(inv_data, np.arange(0.02,20,0.001))

    print(min_peakind)
    plt.figure(figsize=(3,3))
    plt.plot(r,kernel(r))
    sns.kdeplot(K2[N])
    plt.scatter(r[min_peakind],kernel(r[min_peakind]))
#    plt.yscale('log')
    plt.show()
    # Q=input("Peak Num (0 based):")
    Q=0
    ThK2[N]=r[min_peakind[np.int(Q)]]

In [ ]:
ThK1={'CD45': 1.8686868686868687}
ThK2={'CD45': 2.1717171717171717}


In [ ]:
K1CD45Neg=K1[K1.CD45<ThK1['CD45']].copy()
K2CD45Neg=K2[K2.CD45<ThK2['CD45']].copy()


In [ ]:
sns.histplot(K1.CD45,color='b',label='T1',stat='density',element='step',fill=False,)
#sns.histplot(K2.CD45,color='b',label='T2',stat='density')

In [ ]:
sns.kdeplot(K1.CD45,color='b',label='T1')
sns.kdeplot(K1CD45Neg.CD45,color='b',ls='--',label='T1 CD45-')
sns.kdeplot(K2.CD45,c='r',label='T2')
sns.kdeplot(K2CD45Neg.CD45,c='r',ls='--',label='T2 CD45-')
plt.legend(loc='upper left',bbox_to_anchor=(1,1))
plt.yscale('log')
#plt.savefig('Plots/aSMA.png',dpi=200,bbox_inches='tight')

# Normalize using new method on all intercellular markers

In [ ]:
def R(p,x,data,Q,M,M1,M2,M3):
    a=p['a']
    b=p['b']
    d=x.divide(a*M1+(1-a-b)*M2+b*M3,axis=0)
    return (d.std()['H3.3']+d.std()['H4']+d.std()['H3'])**2

def NormalizeNew(data):
    
    params = Parameters()
    params.add('a', value=0.1,min=0,max=1)
    params.add('b', value=0.1,min=0,max=1)
    ddf=data.copy()
    ddf2=data.copy()
    Q=ddf.mean()
    M=(ddf/Q)[['H3.3','H4','H3']].mean(axis=1)
    M1=(ddf/Q)['H3.3']
    M2=(ddf/Q)['H4']
    M3=(ddf/Q)['H3']

    out=minimize(R, params ,args=(ddf, ddf,Q,M,M1,M2,M3),method='cg')
    AA=out.params['a'].value
    BB=out.params['b'].value
    M=M1*AA+M2*(1-AA-BB)+M3*BB
    ddf=ddf.divide(M,axis=0).copy()
    data=ddf
    ddf2[EpiCols]=data[EpiCols]
    data=ddf2.copy()
    del ddf 
    del ddf2
    return data
    
def R2(p,x,data,Q,M,M1,M2):
    a=p['a']
    d=x.divide(a*M1+(1-a)*M2,axis=0)
    return (d.std()['H3.3'])**2+(d.std()['H4'])**2

def NormalizeNew2(data):
    
    params = Parameters()
    params.add('a', value=0.5,min=0.3,max=1)
    ddf=data.copy()
    ddf2=data.copy()
    Q=ddf.mean()
    M=(ddf/Q)[['H3.3','H4']].mean(axis=1)
    M1=(ddf/Q)['H3.3']
    M2=(ddf/Q)['H4']
 
    out=minimize(R2, params ,args=(ddf, ddf,Q,M,M1,M2),method='cg')
    AA=out.params['a'].value

    M=M1*AA+M2*(1-AA)
    ddf=ddf.divide(M,axis=0).copy()
    data=ddf.copy()
    print(data.shape,ddf2.shape)
    ddf2[EpiCols]=data[EpiCols]
    data=ddf2.copy()
    del ddf 
    del ddf2
    return data

In [ ]:
K1.std()

In [ ]:
EpiBCK=EpiCols.copy()
EpiCols=ToNorm.copy()






print("K1")
print(K1.std()['H3.3']+K1.std()['H4'])#+K1.std()['H3'])
K1=NormalizeNew2(K1)
print(K1.std()['H3.3']+K1.std()['H4'])#+K1.std()['H3'])

print("K2")
print(K2.std()['H3.3']+K2.std()['H4'])#+K2.std()['H3'])
K2=NormalizeNew2(K2)
print(K2.std()['H3.3']+K2.std()['H4'])#+K2.std()['H3'])


K1CD45Neg=NormalizeNew2(K1CD45Neg)
K2CD45Neg=NormalizeNew2(K2CD45Neg)

EpiCols=EpiBCK.copy()


In [ ]:
K1.std()

In [ ]:
sns.kdeplot(K1.EpCam)

In [ ]:
K2.columns

In [ ]:
Mean_Core=K1[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1[N]=K1[N]/Mean_Core
    
    
Mean_Core=K2[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2[N]=K2[N]/Mean_Core

In [ ]:
Mean_Core=K1[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1[N]=K1[N]/Mean_Core
    
    
Mean_Core=K2[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2[N]=K2[N]/Mean_Core

Mean_Core=K1CD45Neg[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1CD45Neg[N]=K1CD45Neg[N]/Mean_Core
    
    
Mean_Core=K2CD45Neg[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2CD45Neg[N]=K2CD45Neg[N]/Mean_Core

In [ ]:
sns.kdeplot(K1.CD45)
sns.kdeplot(K2.CD45)

In [ ]:
aaaa=pd.concat([K1]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K1=(K1-m)/s

aaaa=pd.concat([K2]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K2=(K2-m)/s

aaaa=pd.concat([K1CD45Neg]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K1CD45Neg=(K1CD45Neg-m)/s

aaaa=pd.concat([K2CD45Neg]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K2CD45Neg=(K2CD45Neg-m)/s


print(aaaa.std())

params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

In [ ]:
Lines=['Tumor1','Tumor2']

In [ ]:
for M in NamesAll:
    plt.figure()
    sns.kdeplot(K1[M],c='r',label='Tumor 1')
    sns.kdeplot(K2[M],c='g',label='Tumor 2')
   
    plt.legend()
    plt.title('Normalized + Scaled ')
#    plt.savefig('Plots/Hist_Norm_Scaled_'+M+'.png')
    plt.show()

In [ ]:
print(len(K1),len(K2))
print(len(K1CD45Neg),len(K2CD45Neg))

In [ ]:
sns.kdeplot(K1CD45Neg.aSMA)

In [ ]:


K1 = K1CD45Neg.copy()
K2 = K2CD45Neg.copy()
del K2CD45Neg
del K1CD45Neg

# UMAP Tumor 1 - ALL

## Cell Identity

In [ ]:

CAll=pd.concat([K1]).copy()

In [ ]:
# CellIden

In [ ]:
#CellIden.remove('CD45')
X_2d=draw_umap(CAll[CellIden+['p53']],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 Cell Iden UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_CellIdentity_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.1,min_samples=40)

In [ ]:
m=labels!=-1

In [ ]:
sns.histplot(K1[labels!=0].CD45,color='r',element='step',fill=False,stat='density')
sns.histplot(K1.CD45,element='step',fill=False,stat='density')
plt.yscale('symlog')

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
# K1AN=K1AN[:100]

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)

In [ ]:
# scs.inference.nested_model(K1AN)

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2,frameon=True,
               title='Clusters Tumor 1 - Cell Iden Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_CellIden.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based') 
plt.savefig('Plots/T1_CellIden.png')

## Epigenetics Based

In [ ]:
MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 Epigen UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_Epi'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.2,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Epigen Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_Epigen.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Epigen Based') 
plt.savefig('Plots/T1_Epigen.png')

## All Markers

In [ ]:
MRK=[
 'CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:

X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 All Markers UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_All'+NN+'.png',dpi=200,bbox_inches='tight')
    # plt.savefig("Plots/Clust_T1_CellIden.png")

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.18,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - All MRK Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_All.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 All MRK Based') 
plt.savefig('Plots/T1_All.png')

# UMAP Tumor 1 - CD 45 Negative

In [ ]:
K1Bck=K1.copy()
K1=K1CD45Neg.copy()

## Cell Identity

In [ ]:

CAll=pd.concat([K1]).copy()

In [ ]:
#CellIden.remove('CD45')
X_2d=draw_umap(CAll[CellIden],cc=CAll['H4'],min_dist=0.01,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("CD45- Tumor 1 Cell Iden UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_CD45Neg_UMAP_CellIdentity_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.35,min_samples=60)

In [ ]:
m=labels!=-1

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
#sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Cell Iden Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_CellIden_CD45Neg.png")

In [ ]:
with rc_context({'figure.figsize': (4, 4)}):
    sc.pl.umap(K1AN, color=NamesAll+['clust'],ncols=5,vmax='p99.9',vmin='p0.001',
           cmap=plt.cm.seismic,add_outline=True,show=False)
    plt.savefig("Plots/T.pdf",dpi=200,bbox_inches='tight')


In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,15), annot_kws={"size":12}, center=0,yticklabels=True,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based (45-)') 
plt.savefig('Plots/T1_CellIden_CD45Neg.png')

In [ ]:
EP=EpiCols.copy()
EP.remove('H3')
EP.remove('H3.3')
EP.remove('H4')

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[EP]
amin=Mat[EP].min().min()
amax=Mat[EP].max().max()
g=sns.clustermap(Mat[EP].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,15), annot_kws={"size":12}, center=0,yticklabels=True,col_cluster=False,row_cluster=True,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based (45-)') 
plt.savefig('Plots/T1_CellIden_CD45Neg_Epi.png')

In [ ]:
MeanDist(K1[K1.Clust==0],K1[K1.Clust==1],EP)

## Epigenetics Based

In [ ]:
MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("(CD45-) Tumor 1 Epigen UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_Epi_CD45Neg'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.3,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Epigen Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_Epigen_CD45Neg.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[CellIden].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                 annot_kws={"size":8}, center=0,figsize=(10,20),
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Epigen Based (CD45-)') 
plt.savefig('Plots/T1_Epigen_CD45Neg.png')

## All Markers

In [ ]:
MRK=[
 'CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:

X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("(CD45-) Tumor 1 All Markers UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_All_CD45Neg'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.17,min_samples=10)

In [ ]:
m=labels!=-1

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - All MRK Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_All_CD45Neg.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 All MRK Based (CD45-)') 
plt.savefig('Plots/T1_All_CD45Neg.png')

# UMAP Tumor 2

In [ ]:
CAll=pd.concat([K2]).copy()
X_2d=draw_umap(CAll[CellIden],cc=CAll['H4'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
# THESE 3 CELLS WERE BELOW THE 4TH..
CAll.reset_index(inplace=True)

In [ ]:
X_2dB=X_2d.copy()
CAllB=CAll.copy()

In [ ]:
X_2d=X_2dB.copy()
CAll=CAllB.copy()

In [ ]:
for NN in ['CD45']:
    Var=NN
    TSNEVar=NN
    cc=CAllB[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2dB[:,0],X_2dB[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 2 "+TSNEVar)
#    plt.savefig('Plots/Tumor2_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
idx=np.random.choice(CAll.index,replace=False,size=10000)

plt.scatter(X_2d[idx,0],X_2d[idx,1])
X_2d=X_2d[idx]
CAll=CAll.iloc[idx]

In [ ]:
for NN in CellIden:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 2 "+TSNEVar)
#    plt.savefig('Plots/Tumor2_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.11,min_samples=100)

In [ ]:
m=labels!=-1


In [ ]:
K2AN=anndata.AnnData(CAll[m][CellIden],dtype=np.float32)

In [ ]:
sc.pp.neighbors(K2AN)
sc.tl.umap(K2AN)
K2AN.obsm['X_umap']=X_2d[m]

In [ ]:
CAll['Clust']=labels

In [ ]:
K2AN.obs['clust']=CAll[m].Clust.astype('category').values

In [ ]:

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K2AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2,frameon=True,
               title='Clusters Tumor 2', palette=['r','orange','yellow','g','b','indigo','magenta'],show=False)
plt.savefig("Plots/Clust_T2.png")

In [ ]:
Mat=CAll[CAll.Clust!=-1].groupby(by='Clust').mean()[NamesAll]
amin=Mat[CellIden].min().min()
amax=Mat[CellIden].max().max()
g=sns.clustermap(Mat[CellIden].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,10), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
plt.savefig('Plots/T2.png')

In [ ]:
sns.kdeplot(K1.CD45)
sns.kdeplot(K2.CD45)

In [ ]:
end = time.process_time()
print ('total run time =', end-start )